In [1]:
import pandas as pd
import numpy as np
from geopy import distance

In [2]:
MA_district_coordinates = pd.read_excel('data/MA_AP_performance/MA_district_coordinates.xlsx').sort_values(by='District Code').reset_index(drop=True)

uni_coord_R1R2 = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_R1R2_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_Public = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_Public_university_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_Private_nfp = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_private_notforprofit_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_Land_Grant = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_Land_Grant_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)
uni_coord_STEM = pd.read_excel('data/MA_AP_performance/hd2023_coordinate/hd2023_STEM_data_MA_allnearby.xlsx').sort_values(by='INSTNM').reset_index(drop=True)


###########################

def distance_district_univeristy_R1R2(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_R1R2[uni_coord_R1R2['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_Public(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_Public[uni_coord_Public['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_Private_nfp(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_Private_nfp[uni_coord_Private_nfp['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_Land_Grant(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_Land_Grant[uni_coord_Land_Grant['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles

def distance_district_univeristy_STEM(district_code, university_name):
    district_coor = MA_district_coordinates[MA_district_coordinates['District Code'] == district_code][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    university_coor = uni_coord_STEM[uni_coord_STEM['INSTNM'] == university_name][['LONGITUD', 'LATITUDE']].to_numpy()[0][::-1]
    return distance.distance(district_coor, university_coor).miles    ## return unit is miles


###########################

epsilon_soften_factor = 10.0  ## [miles]

def inverse_distance_weight(distance_miles):
    return 1.0 / ( (distance_miles/epsilon_soften_factor) + 1 )

def enrollment_inverse_distance_weight(distance_miles, num_annual_enrollment):
    ## num_annual_enrollment is the number of annual enrollment
    return int(num_annual_enrollment) * 1.0 / ( (distance_miles/epsilon_soften_factor) + 1 )

def dormbed_inverse_distance_weight(distance_miles, num_dorm_bed):
    return int(num_dorm_bed) * 1.0 / ( (distance_miles/epsilon_soften_factor) + 1 )

# First, find the school districts that appear in all five year data

In [66]:
MA_distance_dict = pd.read_excel('data/MA_AP_performance/AP_performance_18_22.xlsx', sheet_name='20%s-%s'%(18, 18+1))
for year in [19, 20, 21, 22]:
    MA_distance_dict_next = pd.read_excel('data/MA_AP_performance/AP_performance_18_22.xlsx', sheet_name='20%s-%s'%(year, year+1))
    MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_distance_dict['District Code'])]

MA_median_income = pd.read_excel('data/MA_AP_performance/MA_median_household_income.xlsx')
MA_district_population = pd.read_excel('data/MA_AP_performance/MA_district_population.xlsx')

MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_district_coordinates['District Code'])]
MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_median_income['District Code'])]
MA_distance_dict = MA_distance_dict[MA_distance_dict['District Code'].isin(MA_district_population['District Code'])]
MA_distance_dict = MA_distance_dict.sort_values(by='District Code').reset_index(drop=True)

#### delete redundant columns in the AP_performance excel sheet 
MA_distance_dict = MA_distance_dict.drop(columns=['Tests Taken', 'Score=1', 'Score=1', 'Score=2', 'Score=3', 'Score=4', 'Score=5', '% Score 1-2', '% Score 3-5'])
MA_distance_dict = MA_distance_dict.sort_values(by='District Code').reset_index(drop=True)

In [67]:
MA_distance_dict

,District Name,District Code
0,Abington,10000
1,Agawam,50000
2,Amesbury,70000
3,Andover,90000
4,Arlington,100000
...,...,...
217,Tantasqua,7700000
218,Triton,7730000
219,Wachusett,7750000
220,Quaboag Regional,7780000


# Second, calculate all distance-related quantities
### The following cell will take about 2 minutes to run

In [74]:
#### 1. Calculate distance to R1R2 universities
distance_arr = []
enll_arr = []
dormbed_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    distance_enroll_dormbed_arr = []
    
    for university_name in uni_coord_R1R2['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_R1R2(district_code, university_name)
        num_annual_enrollment = uni_coord_R1R2[uni_coord_R1R2['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_R1R2[uni_coord_R1R2['INSTNM'] == university_name]['Number of dorm beds'].values[0]

        distance_enroll_dormbed_arr += [[distance_ij, num_annual_enrollment, num_dorm_bed]]
        
    distance_enroll_dormbed_arr = sorted(distance_enroll_dormbed_arr, key=lambda x: x[0])
    distance_enroll_dormbed_arr = np.array( distance_enroll_dormbed_arr[:5] ) 

    distance_arr += [np.sum(distance_enroll_dormbed_arr[:,0])]
    enll_arr     += [np.sum(distance_enroll_dormbed_arr[:,1])]
    dormbed_arr  += [np.sum(distance_enroll_dormbed_arr[:,2])]
    
MA_distance_dict['Shortest Distance to R1R2'] = distance_arr
MA_distance_dict['Number of Annual Enrollment R1R2'] = enll_arr
MA_distance_dict['Number of Dorm Beds R1R2'] = dormbed_arr


#### 2. Calculate distance to Public universities
distance_arr = []
enll_arr = []
dormbed_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    distance_enroll_dormbed_arr = []
    
    for university_name in uni_coord_Public['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_Public(district_code, university_name)
        num_annual_enrollment = uni_coord_Public[uni_coord_Public['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_Public[uni_coord_Public['INSTNM'] == university_name]['Number of dorm beds'].values[0]

        distance_enroll_dormbed_arr += [[distance_ij, num_annual_enrollment, num_dorm_bed]]
        
    distance_enroll_dormbed_arr = sorted(distance_enroll_dormbed_arr, key=lambda x: x[0])
    distance_enroll_dormbed_arr = np.array( distance_enroll_dormbed_arr[:5] ) 

    distance_arr += [np.sum(distance_enroll_dormbed_arr[:,0])]
    enll_arr     += [np.sum(distance_enroll_dormbed_arr[:,1])]
    dormbed_arr  += [np.sum(distance_enroll_dormbed_arr[:,2])]
    
MA_distance_dict['Shortest Distance to Public'] = distance_arr
MA_distance_dict['Number of Annual Enrollment Public'] = enll_arr
MA_distance_dict['Number of Dorm Beds Public'] = dormbed_arr


#### 3. Calculate distance to Private nfp universities
distance_arr = []
enll_arr = []
dormbed_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    distance_enroll_dormbed_arr = []
    
    for university_name in uni_coord_Private_nfp['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_Private_nfp(district_code, university_name)
        num_annual_enrollment = uni_coord_Private_nfp[uni_coord_Private_nfp['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_Private_nfp[uni_coord_Private_nfp['INSTNM'] == university_name]['Number of dorm beds'].values[0]

        distance_enroll_dormbed_arr += [[distance_ij, num_annual_enrollment, num_dorm_bed]]
        
    distance_enroll_dormbed_arr = sorted(distance_enroll_dormbed_arr, key=lambda x: x[0])
    distance_enroll_dormbed_arr = np.array( distance_enroll_dormbed_arr[:5] ) 

    distance_arr += [np.sum(distance_enroll_dormbed_arr[:,0])]
    enll_arr     += [np.sum(distance_enroll_dormbed_arr[:,1])]
    dormbed_arr  += [np.sum(distance_enroll_dormbed_arr[:,2])]
    
MA_distance_dict['Shortest Distance to Private'] = distance_arr
MA_distance_dict['Number of Annual Enrollment Private'] = enll_arr
MA_distance_dict['Number of Dorm Beds Private'] = dormbed_arr


#### 4. Calculate distance to Land Grant universities
distance_arr = []
enll_arr = []
dormbed_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    distance_enroll_dormbed_arr = []
    
    for university_name in uni_coord_Land_Grant['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_Land_Grant(district_code, university_name)
        num_annual_enrollment = uni_coord_Land_Grant[uni_coord_Land_Grant['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_Land_Grant[uni_coord_Land_Grant['INSTNM'] == university_name]['Number of dorm beds'].values[0]

        distance_enroll_dormbed_arr += [[distance_ij, num_annual_enrollment, num_dorm_bed]]
        
    distance_enroll_dormbed_arr = sorted(distance_enroll_dormbed_arr, key=lambda x: x[0])
    distance_enroll_dormbed_arr = np.array( distance_enroll_dormbed_arr[:5] ) 

    distance_arr += [np.sum(distance_enroll_dormbed_arr[:,0])]
    enll_arr     += [np.sum(distance_enroll_dormbed_arr[:,1])]
    dormbed_arr  += [np.sum(distance_enroll_dormbed_arr[:,2])]
    
MA_distance_dict['Shortest Distance to Land Grant'] = distance_arr
MA_distance_dict['Number of Annual Enrollment Land Grant'] = enll_arr
MA_distance_dict['Number of Dorm Beds Land Grant'] = dormbed_arr



#### 5. Calculate distance to STEM universities
distance_arr = []
enll_arr = []
dormbed_arr = []

for district_code in MA_distance_dict['District Code'].tolist():
    
    distance_enroll_dormbed_arr = []
    
    for university_name in uni_coord_STEM['INSTNM'].tolist():
        
        distance_ij           = distance_district_univeristy_STEM(district_code, university_name)
        num_annual_enrollment = uni_coord_STEM[uni_coord_STEM['INSTNM'] == university_name]['Annual enrollment'].values[0]
        num_dorm_bed          = uni_coord_STEM[uni_coord_STEM['INSTNM'] == university_name]['Number of dorm beds'].values[0]

        distance_enroll_dormbed_arr += [[distance_ij, num_annual_enrollment, num_dorm_bed]]
        
    distance_enroll_dormbed_arr = sorted(distance_enroll_dormbed_arr, key=lambda x: x[0])
    distance_enroll_dormbed_arr = np.array( distance_enroll_dormbed_arr[:5] ) 

    distance_arr += [np.sum(distance_enroll_dormbed_arr[:,0])]
    enll_arr     += [np.sum(distance_enroll_dormbed_arr[:,1])]
    dormbed_arr  += [np.sum(distance_enroll_dormbed_arr[:,2])]
    
MA_distance_dict['Shortest Distance to STEM'] = distance_arr
MA_distance_dict['Number of Annual Enrollment STEM'] = enll_arr
MA_distance_dict['Number of Dorm Beds STEM'] = dormbed_arr


#### save this years data in a excel file
MA_distance_dict.to_excel('data/MA_AP_performance/AP_data_combined/MA_closest_university_dict.xlsx', index=False)
MA_distance_dict

,District Name,District Code,Shortest Distance to R1R2,Number of Annual Enrollment R1R2,Number of Dorm Beds R1R2,Shortest Distance to Public,Number of Annual Enrollment Public,Number of Dorm Beds Public,Shortest Distance to Private,Number of Annual Enrollment Private,Number of Dorm Beds Private,Shortest Distance to Land Grant,Number of Annual Enrollment Land Grant,Number of Dorm Beds Land Grant,Shortest Distance to STEM,Number of Annual Enrollment STEM,Number of Dorm Beds STEM
0,Abington,10000,85.871580,119854.0,35215.0,53.142635,49951.0,4381.0,54.807418,8814.0,4318.0,282.041583,95123.0,38339.0,85.871580,119854.0,35215.0
1,Agawam,50000,197.381561,92043.0,35371.0,50.184574,27240.0,2492.0,31.193375,13378.0,4361.0,322.184406,114903.0,45492.0,192.890688,78411.0,29795.0
2,Amesbury,70000,129.122131,108676.0,36275.0,78.143602,37666.0,6862.0,78.353411,19001.0,6937.0,259.312439,109796.0,45744.0,129.122131,108676.0,36275.0
3,Andover,90000,87.621089,95272.0,31466.0,66.054439,62268.0,7069.0,43.096023,17169.0,5810.0,245.167702,109796.0,45744.0,87.621089,95272.0,31466.0
4,Arlington,100000,22.444526,125226.0,41891.0,37.681456,47078.0,1996.0,15.519025,56076.0,16038.0,244.153352,101086.0,40314.0,22.444526,125226.0,41891.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Tantasqua,7700000,129.506095,87940.0,34698.0,105.271614,56520.0,13617.0,80.618211,13228.0,7050.0,252.276889,114903.0,45492.0,106.128768,78411.0,29795.0
218,Triton,7730000,128.540096,108676.0,36275.0,80.271605,49581.0,7069.0,55.092878,19109.0,6800.0,260.145671,101086.0,40314.0,128.540096,108676.0,36275.0
219,Wachusett,7750000,115.470378,75910.0,25492.0,54.390130,35480.0,3314.0,33.451210,13691.0,8215.0,251.426581,114903.0,45492.0,89.170757,54996.0,20705.0
220,Quaboag Regional,7780000,136.596018,83536.0,32745.0,103.544704,62483.0,15592.0,90.160378,13659.0,6374.0,257.335352,114903.0,45492.0,110.200232,78411.0,29795.0


# Load AP data from 2018 to 2022, then concat the distance data (shown above) to the AP data.

In [75]:
for year in [18, 19, 20, 21, 22]:

    MA_median_income = pd.read_excel('data/MA_AP_performance/MA_median_household_income.xlsx')
    MA_district_population = pd.read_excel('data/MA_AP_performance/MA_district_population.xlsx')
    MA_expenditure_year = pd.read_excel('data/MA_AP_performance/PerPupilExpenditures/PerPupilExpenditures_20%s.xlsx'%year)
    
    
    MA_AP_performance_year = pd.read_excel('data/MA_AP_performance/AP_performance_18_22.xlsx', sheet_name='20%s-%s'%(year, year+1))
    MA_AP_performance_year = MA_AP_performance_year[MA_AP_performance_year['District Code'].isin(MA_distance_dict['District Code'])]
    MA_AP_performance_year = MA_AP_performance_year[MA_AP_performance_year['District Code'].isin(MA_expenditure_year['District Code'])]
    MA_AP_performance_year = MA_AP_performance_year.sort_values(by='District Code').reset_index(drop=True)
    
    
    
    #### delete redundant columns in the AP_performance excel sheet 
    MA_all_data_year = MA_AP_performance_year.drop(columns=['Score=1', 'Score=1', 'Score=2', 'Score=3', 'Score=4', 'Score=5', '% Score 1-2'])
    MA_all_data_year.insert(2, 'Year', [2000+year] * MA_all_data_year.shape[0])
    MA_all_data_year = MA_all_data_year.sort_values(by='District Code').reset_index(drop=True)
    
    
    #### Add a colum for district population
    MA_district_population = MA_district_population[MA_district_population['District Code'].isin(MA_all_data_year['District Code'])]
    MA_district_population = MA_district_population.sort_values(by='District Code').reset_index(drop=True)
    MA_all_data_year['Population'] = MA_district_population['Population']
    
    #### Add a colum for district median household income
    MA_median_income = MA_median_income[MA_median_income['District Code'].isin(MA_all_data_year['District Code'])]
    MA_median_income = MA_median_income.sort_values(by='District Code').reset_index(drop=True)
    MA_all_data_year['Median Household Income'] = MA_median_income['Median Household Income']
    
    #### Add a colum for district total expenditure per pupil
    MA_expenditure_year = MA_expenditure_year[MA_expenditure_year['District Code'].isin(MA_all_data_year['District Code'])]
    MA_expenditure_year = MA_expenditure_year.sort_values(by='District Code').reset_index(drop=True)
    expenditure_arr = MA_expenditure_year['Total Expenditures per Pupil'].tolist()
    expenditure_arr = np.array([float(str(x).replace('$', '').replace(',', '').strip()) if isinstance(x, str) else float(x) for x in expenditure_arr])
    MA_all_data_year['Total Expenditures per Pupil'] = expenditure_arr
    
    #### save this years data in a excel file
    MA_all_data_year = pd.concat([MA_all_data_year, MA_distance_dict.iloc[:, 2:]], axis=1)
    MA_all_data_year.to_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_20%s.xlsx'%year, index=False)

# Combine all five year data in one single Excel file

In [76]:
data18 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2018.xlsx').sort_values(by='District Code')
data19 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2019.xlsx').sort_values(by='District Code')
data20 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2020.xlsx').sort_values(by='District Code')
data21 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2021.xlsx').sort_values(by='District Code')
data22 = pd.read_excel('data/MA_AP_performance/AP_data_combined/AP_data_combined_2022.xlsx').sort_values(by='District Code')

combined_data_18_22 = pd.concat([data18, data19, data20, data21, data22], ignore_index=True)

combined_data_18_22.to_csv("data/MA_AP_performance/AP_data_combined_18_22.csv", index=False)

with pd.ExcelWriter("data/MA_AP_performance/AP_data_combined_18_22.xlsx") as writer:
    combined_data_18_22.to_excel(writer, sheet_name="2018-22", index=False)
    data18.to_excel(writer, sheet_name="2018-19", index=False)
    data19.to_excel(writer, sheet_name="2019-20", index=False)
    data20.to_excel(writer, sheet_name="2020-21", index=False)
    data21.to_excel(writer, sheet_name="2021-22", index=False)
    data22.to_excel(writer, sheet_name="2022-23", index=False)